In [7]:
import base64
import requests
import json
import pickle
import json
TEST_NUM=101
responseList=[]
answerList=[]
with open ('../outlist.pkl','rb') as f:
	output_list=pickle.load(f)
url = '' ##Self-hosted AWS server
headers = {
		'Content-Type': 'application/json'
	}
with open ('../data_test.json','r') as f:
	test_dataset=json.load(f)
for index,ouput in enumerate(output_list):
	if index<101:
		data=test_dataset[index]
		answerList.append(data['caption_L2L3'])
		img_id=data['img_id']
		cutData=data['datatable']
		spec_path='../vl_spec/'+img_id+'.vl.json'
		with open(spec_path, 'r') as f:
			spec = json.load(f)
		payload = {
			"inputs": f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. ### Instruction: {spec} ### Input: {cutData} ### Response:",
			"parameters": {
				"max_tokens": 4000,
				"max_new_tokens": 100,
				"temperature": 0.1,
				"use_cache": "False",
				"model_max_length": 4000
			}
		}
		payload_json = json.dumps(payload)
		response = requests.post(url, headers=headers, data=payload_json)
		if response.status_code == 200:
			responseList.append(response.json()['generated_text'])
		else:
			print(f"Request failed with status code {response.status_code}")
	else:
		break


In [ ]:
responseList[:10]

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
scoreList=[]
for i in range(TEST_NUM):
	# if i>87:
	# 	query_embedding = model.encode(responseList[i-1])
	# 	passage_embedding = model.encode(answerList[i])
	# else:
	query_embedding = model.encode(responseList[i])
	passage_embedding = model.encode(answerList[i])
	scoreList.append(float(util.dot_score(query_embedding, passage_embedding)))
len(scoreList)

In [ ]:
import numpy as np
np.average(scoreList[:100])

In [13]:
with open('llama-score.pkl', 'wb') as file:
    pickle.dump(scoreList, file)